# Setup

In [ ]:
!pip install -q pyspark==3.5.5 spark-nlp==5.5.3
!pip install skl2onnx
!pip install onnxruntime-gpu
!pip install tldextract

# Can also use the script below but the pyspark version is incompatible.
# !wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.2 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incomp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sparknlp

spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 5.5.3
Apache Spark version: 3.5.5


In [ ]:
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
import ast
from sparknlp.annotator import StopWordsCleaner
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sparknlp.annotator import StopWordsCleaner
from pyspark.ml import Pipeline
from sparknlp.base import LightPipeline
import os
import pandas as pd
import json
import joblib
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
import numpy

In [ ]:
sample_text = """
Most taxpayers will get a check from the $2tn economic package, the largest in US history

US coronavirus stimulus checks: are you eligible and how much will you get?

The US has agreed on a $2tn stimulus package, the largest economic stimulus in US history, in response to the economic impacts of Covid-19. While corporations will be the biggest recipients of the bailout, some of that money will be paid directly to Americans hit by the pandemic.

Most taxpayers will get a check in the mail, while those directly affected by the economic effects of Covid-19 are slated to receive robust government support.

Here’s what we know so far about how the new stimulus package will directly affect Americans in the coming weeks:

Who’s eligible for a check from the government?

Congress will spend about $250bn for checks up to $1,200 per person that will go directly to taxpayers.

To be eligible for the full amount, a person’s most recently filed tax return must show that they made $75,000 or under. For couples, who can receive a maximum of $2,400, the cutoff is $150,000.

If a person makes more than $75,000, the amount given goes down incrementally by $5 for every $100 increase in salary. So a person who makes $85,000 would get $700 while a person who makes $95,000 would get $200.

If a person makes above $99,000, or a couple makes above $198,000, no check will be given.

There are some requirements and exceptions. Those getting a check must be living and working in the US and have a valid social security number. If a person is listed as a dependent on their parents’ tax return – the case for many college students – they are excluded.

The Tax Foundation, a DC-based thinktank, estimates that 93.6% of Americans will be eligible for a check coming from the stimulus package.

What about parents?

Taxpayers will be given $500 per child 16 or under listed as a dependent on their latest tax return.

When will I get this money?

That is still unclear. Experts say that Americans will probably not get the money until May, given how long it takes the Internal Revenue Service (IRS) to send out checks. The treasury secretary, Steven Mnuchin, said those whose bank account information is already with the IRS from previous tax returns can expect a payment “within three weeks”.

For those who still need to give the IRS their bank account information, a web-based portal will be opening soon to allow people to provide the IRS with their information. The IRS says this is to make sure people get their checks immediately, instead of sending them through the mail.

This is not the first time the government has sent checks to Americans. The federal government gave up to $300 in 2001 and $600 in 2008 to taxpayers who met a certain income bracket to similarly stimulate the economy.

What if a person did not file their 2018 or 2019 return?

The IRS recommends people file their 2018 or 2019 tax return as soon as possible to get the payment. A check will be sent to any qualified person so long as they file their return within 2020.

What if a person does not file their taxes?

A person may still be eligible even if they do not file the taxes. The IRS says people who typically do not file taxes – low-income taxpayers and some veterans – will need to file a “simple tax return” in order to get the payment. Social security beneficiaries will not need to fill out the tax return, even if they do not file their taxes, to get the payment.

Does the package help unemployed workers?

Yes, specifically the bill will increase unemployment insurance by $600 for 13 weeks – about four months – for every person, added to the existing unemployment compensation a person gets from their state’s program.

The length and amount of compensation varies from state to state. A majority of states providing a maximum of 26 weeks of compensation, while average weekly compensation ranges from 20% of a person’s wage to just over 50%.

What about freelance and gig workers?

While unemployment insurance typically does not cover people who are self-employed – freelancers, contractors and gig workers – the bill comes with a “pandemic unemployment assistance” measure that will extend insurance to those workers.

Play Video 2:25 'The universe is collapsing': Bernie Sanders mocks Republicans over coronavirus aid – video

Have there been any changes to paid leave?

Not in this bill. Earlier last week, Donald Trump signed the Families First Coronavirus Response Act, a bill worth about $100bn meant to expand paid sick leave and emergency paid leave, but it came with major loopholes. Companies with over 500 employees were not mentioned in the bill, while companies with under 50 employees can apply for exemptions.

The bill mandates 10 days of fully paid sick leave for employees of companies with 500 employees or less. Parents of those companies affected by their children’s school closing and those leaving for medical reasons can get 12 weeks of pay at 67% of their salary.

Will there be anything to offset healthcare costs related to Covid-19?

Again, not in this bill. The Families First Coronavirus Response Act included a measure that mandated all Covid-19 testing is free, but treatment for any symptoms (there is currently no cure for the illness) still comes at a cost.

A few states have reopened enrollment for their health insurance programs to allow those concerned about costs to enroll, and three major health insurance companies said they will be waiving any high costs for treatment, but there are still stories of people getting bills for as much as $34,000 to cover treatment of the virus’s symptoms.
"""

In [ ]:
def init_clean_text_pipeline(text):
  nlp_pipeline = PretrainedPipeline("explain_document_ml", lang="en")
  stopwords_cleaner = StopWordsCleaner().pretrained('stopwords_iso', 'en') \
    .setInputCols(["token"]) \
    .setOutputCol("cleanTokens")
  default_stages = nlp_pipeline.model.stages
  custom_pipeline = Pipeline(stages=default_stages + [stopwords_cleaner])
  pipeline_model = custom_pipeline.fit(spark.createDataFrame([[text]]).toDF("text"))
  light_model = LightPipeline(pipeline_model, parse_embeddings=True)
  return light_model


clean_text_pipeline = init_clean_text_pipeline(sample_text)

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]
stopwords_iso download started this may take some time.
Approximate size to download 2.1 KB
[OK!]


In [ ]:
# Define a function that takes an original text and returns a cleaned version
def clean_text(text, light_model=clean_text_pipeline, remove_stop_words=False, fix_spelling=False, join=True, delimiter=" "):
  result = light_model.annotate(text)
  output_col = 'cleanTokens' if remove_stop_words else ('spell' if fix_spelling else 'token')
  return delimiter.join(result[output_col]) if join else result[output_col]

In [ ]:
# Example 1: returning original with space characters as delimiter
text = clean_text("Hello, my name is Jitrapon. I am studying Capstone. This is a very fun project! Thre is some spellling errors!")
assert(text == 'Hello , my name is Jitrapon . I am studying Capstone . This is a very fun project ! Thre is some spellling errors !')

# Example 2: returning text with space characters as delimiter but also fix spelling
text = clean_text("Hello, my name is Jitrapon. I am studying Capstone. This is a very fun project! Thre is some spellling errors!", fix_spelling=True)
assert(text == 'Hello , my name is Jitrapon . I am studying Capstone . This is a very fun project ! There is some spelling errors !')

# Example 3: returning a list instead
text_list = clean_text("Hello, my name is Jitrapon. I am studying Capstone. This is a very fun project!", join=False)
assert(text_list == ['Hello',
 ',',
 'my',
 'name',
 'is',
 'Jitrapon',
 '.',
 'I',
 'am',
 'studying',
 'Capstone',
 '.',
 'This',
 'is',
 'a',
 'very',
 'fun',
 'project',
 '!'])

# Example 4: remove all stopwords
text = clean_text("Hello, my name is Jitrapon. I am studying Capstone. This is a very fun project!", remove_stop_words=True)
assert(text == 'Hello , Jitrapon . studying Capstone . fun project !')

# Example 5: using comma as a delimiter
text = clean_text("Hello, my name is Jitrapon. I am studying Capstone. This is a very fun project!", delimiter=",")
assert(text == 'Hello,,,my,name,is,Jitrapon,.,I,am,studying,Capstone,.,This,is,a,very,fun,project,!')

In [ ]:
# Load the sentence-biase classifier
from pyspark.ml import PipelineModel

from google.colab import drive
drive.mount('/content/gdrive')

path = F"/content/gdrive/MyDrive/spark_models/sentence_bias_model"
sentence_bias_model = PipelineModel.load(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Load the source_bias_score.csv to Pandas
path = F"/content/gdrive/MyDrive/spark_dataframes/source_bias_score.csv"
source_bias_score_df = pd.read_csv(path)
source_bias_score_df.drop(columns=['Unnamed: 0'], inplace=True)
source_bias_score_df.head()

,domain,source,count,source_encoded,source_bias_score
0,aljazeera,Al Jazeera,104,4.0,0.5
1,allsides,National Review,3,108.0,1.5
2,apnews,Associated Press,235,14.0,0.0
3,axios,Axios,62,15.0,0.5
4,bbc,BBC News,603,16.0,1.0


In [ ]:
from tldextract import extract
import numpy as np


# Define a function that extracts the domain from the URL, then returns the corresponding 'source', 'source_encoded', and 'source_bias_score'
def convert_url_to_source(url):
  ext = extract(url)
  # Return the row in source_bias_score_df where domain equals ext.domain
  return source_bias_score_df[source_bias_score_df['domain'] == ext.domain]


assert(0.5 == convert_url_to_source('www.cnn.com')['source_bias_score'].values[0])

In [ ]:
# Load test_pd_df_final from csv
path = F"/content/gdrive/MyDrive/spark_dataframes/test_pd_df_final.csv"
test_pd_df_final = pd.read_csv(path)
test_pd_df_final.head()

,Unnamed: 0,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID,source_bias_score,source_encoded,avg_sentence_bias_score
0,0,palestine,Breitbart News,2.0,http://www.breitbart.com/big-journalism/2015/0...,"Obama, Media Lied About Netanyahu and the Pale...",2015-03-20,Ben Shapiro,"For the last several days , the entire Western...","For the last several days, the entire Western ...",www.breitbart.com,right,uI2K5BSCO0LaWuKb,2.0,8.0,0.718750
1,1,federal_budget,ABC News,0.0,http://abcnews.go.com/blogs/politics/2013/03/w...,Will Fractured House Republicans Unite on Budget?,NaN,John Parkinson,"Tomorrow morning , House Budget Chairman Paul ...","Jacquelyn Martin/AP Photo\n\nTomorrow morning,...",www.abcnews.go.com,left,zeSfcxLilzR0R1Dt,0.5,0.0,0.450000
2,2,fbi,Reuters,1.0,https://www.reuters.com/article/us-usa-trump-r...,Trump blocks release of Russia memo drafted by...,2018-02-10,Ayesha Rascoe,WASHINGTON ( ███ ) - President Donald Trump on...,WASHINGTON (Reuters) - President Donald Trump ...,www.reuters.com,center,ru8kAxfdPzZc11le,1.0,23.0,0.366667
3,3,politics,CBN,2.0,http://www1.cbn.com/cbnnews/politics/2019/febr...,Trump's North Korea Peace Talks Share Spotligh...,2019-02-25,NaN,"President Donald Trump arrives in Hanoi , Viet...","President Donald Trump arrives in Hanoi, Vietn...",www1.cbn.com,right,TTt5FFmsW6ng3fsC,2.0,10.0,0.400000
4,4,media_bias,Breitbart News,2.0,http://www.breitbart.com/big-government/2017/0...,Six Times President Trump Upset Jared Kushner ...,2017-08-20,Adam Shaw,President Trump ’ s fiery response to the even...,President Trump’s fiery response to the events...,www.breitbart.com,right,gYKaZGjLCYdEghTZ,2.0,8.0,0.620690


In [ ]:
# Load the TF-IDF vectorizer
path = F"/content/gdrive/MyDrive/spark_models/tfidf_vectorizer.pkl"
tfidf_vectorizer = joblib.load(path)

In [ ]:
# Load the Scaler
path = F"/content/gdrive/MyDrive/spark_models/scaler.pkl"
scaler = joblib.load(path)

In [ ]:
# Define a function that preprocesses all the columns to be predicted
def preprocess_test_columns(df=test_pd_df_final, vectorizer=tfidf_vectorizer, scaler=scaler, addition_features=True):
  X = df["content"]
  y = df["bias"]

  # Create a TF-IDF vectorizer from the text from the fitted vectorizer
  X_text = vectorizer.transform(df['content'])

  if addition_features:

    # Prepare the additional numeric features.
    additional_features = df[['avg_sentence_bias_score', 'source_encoded', 'source_bias_score']].values

    # Standardize these features to put them on a similar scale.
    X_additional_scaled = scaler.transform(additional_features)

    # Convert the additional features to a sparse matrix.
    X_additional_sparse = csr_matrix(X_additional_scaled)

    # Combine the TF-IDF features with the additional features.
    X_final = hstack([X_text, X_additional_sparse])
  else:
    X_final = X_text

  # Get the target variable.
  y = df['bias']

  return (X_final, y)

In [ ]:
# Define a function that prints out accuracy report
def print_accuracy_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)

  print("Accuracy: {:.3f}\n".format(accuracy))
  print(report)

In [ ]:
# Load the saved models
# https://onnx.ai/sklearn-onnx/auto_examples/plot_convert_model.html
import onnx
import onnxruntime as rt


# Preprocess the test dataset
X_test, y_test = preprocess_test_columns()

# Test prediction with ONNX runtime
def predict(X_test, model_path):
  sess = rt.InferenceSession(model_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
  input_name = sess.get_inputs()[0].name
  label_name = sess.get_outputs()[0].name
  pred_onx = sess.run([label_name], {input_name: (X_test.astype(numpy.float32)).toarray()})[0]
  return pred_onx

In [ ]:
# Make sure that the prediction is what we expect
print_accuracy_report(y_test, predict(X_test, "/content/gdrive/MyDrive/spark_models/logreg.onnx"))

Accuracy: 0.869

              precision    recall  f1-score   support

         0.0       0.83      0.97      0.89       402
         1.0       0.71      0.73      0.72       298
         2.0       1.00      0.87      0.93       599

    accuracy                           0.87      1299
   macro avg       0.85      0.86      0.85      1299
weighted avg       0.88      0.87      0.87      1299



In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql.types import StringType, IntegerType


# Setup pipeline to extract sentences out of each article
document = DocumentAssembler() \
    .setInputCol('content') \
    .setOutputCol("document")

# Using setExplodeSentences(True) will create one row per sentence.
sentence_detector = SentenceDetectorDLModel() \
    .pretrained("sentence_detector_dl", "en") \
    .setInputCols("document") \
    .setOutputCol("sentence") \
    .setExplodeSentences(True)

sentence_detector_pipeline = Pipeline(
    stages=[
        document,
        sentence_detector
])

# Define a UDF to extract the sentence from each annotation in an array.
def extract_sentence_text(annotations):
    if annotations is None:
        return []
    for ann in annotations:
      return ann.result

extract_sentence_text_udf = udf(extract_sentence_text, StringType())

# Define a UDF to extract the bias value
def extract_result(result):
    return int(result[0].get)

extract_result_udf = udf(extract_result, IntegerType())

# To use, uncomment
# exploded_test_df = sentence_detector_pipeline.fit(test_df).transform(test_df)
# exploded_test_df = exploded_test_df.withColumn("text", extract_sentence_text_udf(col("sentence")))

# Use the sentence-level bias classifier to predict the bias
# exploded_test_df = sentence_bias_model.transform(exploded_test_df)

# exploded_test_df = exploded_test_df.withColumn("sentence_bias_score", extract_sentence_text_udf(col("class")))
# exploded_test_df.select('ID', 'text', 'sentence_bias_score').show(truncate=False)

# Compute the average bias score
# agg_test_df = exploded_test_df.groupBy("ID") \
    # .agg(mean(col("sentence_bias_score")).alias("avg_sentence_bias_score"))

# Retrieve the average bias score

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [ ]:
from pyspark.sql.functions import avg, round, col, mean


# Define a function that takes in the preprocessed text
# and returns the computed average bias score for the article
def compute_avg_sentence_bias_score(text, debug=False):
  # Create a Spark DataFrame
  test_df = spark.createDataFrame([[text]], ["content"])

  # Extract sentences into a new column called 'text'
  exploded_test_df = sentence_detector_pipeline.fit(test_df).transform(test_df)
  exploded_test_df = exploded_test_df.withColumn("text", extract_sentence_text_udf(col("sentence")))

  # Use the sentence-level bias classifier to predict the bias
  exploded_test_df = sentence_bias_model.transform(exploded_test_df)
  exploded_test_df = exploded_test_df.withColumn("sentence_bias_score", extract_sentence_text_udf(col("class")))

  if debug:
    exploded_test_df.select('text', 'sentence_bias_score').show(truncate=False)

  # Compute the average bias score
  agg_test_df = exploded_test_df\
      .agg(mean(col("sentence_bias_score")).alias("avg_sentence_bias_score"))

  # Retrieve the average bias score
  return agg_test_df.select("avg_sentence_bias_score").collect()[0][0]

In [ ]:
%%time

assert(0.14285714285714285 == compute_avg_sentence_bias_score(sample_text, True))

+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|text                                                                                                                                                           |sentence_bias_score|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|Most taxpayers will get a check from the $2tn economic package, the largest in US history                                                                      |0                  |
|US coronavirus stimulus checks: are you eligible and how much will you get?                                                                                    |0                  |
|The US has agreed on a $2tn stimulus package, the largest economic stimulus in US history

# Input

In [ ]:
model_path = "/content/gdrive/MyDrive/spark_models/logreg.onnx"
url = "https://www.cnbc.com/2025/04/02/elon-musk-trump-doge-tesla-white-house-wisconsin-supreme-court.html"
text = """
President Donald Trump told Cabinet members last week that billionaire Elon Musk will leave his administration role in the coming months, NBC News reported Wednesday.

A senior government official told NBC that Musk — whose DOGE team is engaged in a controversial effort to slash federal spending — would leave at the end of a 130-day stint as a special government employee.

That specific designation, SGE, caps a person’s workdays per year at 130 days. Because Musk started on Jan. 20, his SGE cap will be reached at the end of May.

Politico reported earlier Wednesday that the Tesla
 CEO will soon be leaving the Trump administration. Tesla shares rose after that story was published.

White House press secretary Karoline Leavitt, in a tweet, called that report “garbage.”

“Elon Musk and President Trump have both publicly stated that Elon will depart from public service as a special government employee when his incredible work at DOGE is complete,” Leavitt tweeted.

Musk replied to Leavitt’s tweet, writing, “Yeah, fake news.”

Trump on Monday told reporters, “I think he’s been amazing, but I also think he’s got a big company to run ... And at some point, he’s going to be going back. He wants to.”

“I’d keep him as long as I could keep him,” the president said.

But in a March 10 interview with Fox Business, host Larry Kudlow asked Musk, “You going to go another year?”

Musk replied, “Yeah, I think so.”

Musk was the biggest financial backer of Trump’s 2024 campaign.

On Tuesday night, Brad Schimel, the conservative candidate bankrolled by Musk and backed by Trump, lost in the Wisconsin Supreme Court election.

"""

# Full Pipeline

In [ ]:
# Clean text
text = clean_text(text)
text

'President Donald Trump told Cabinet members last week that billionaire Elon Musk will leave his administration role in the coming months , NBC News reported Wednesday . A senior government official told NBC that Musk — whose DOGE team is engaged in a controversial effort to slash federal spending — would leave at the end of a 130-day stint as a special government employee . That specific designation , SGE , caps a person’s workdays per year at 130 days . Because Musk started on Jan . 20 , his SGE cap will be reached at the end of May . Politico reported earlier Wednesday that the Tesla CEO will soon be leaving the Trump administration . Tesla shares rose after that story was published . White House press secretary Karoline Leavitt , in a tweet , called that report “garbage . ” “Elon Musk and President Trump have both publicly stated that Elon will depart from public service as a special government employee when his incredible work at DOGE is complete,” Leavitt tweeted . Musk replied t

In [ ]:
%%time

# Compute average sentence bias score
avg_sentence_bias_score = compute_avg_sentence_bias_score(text)
avg_sentence_bias_score

0.16666666666666666

In [ ]:
# From the URL, convert to one of the sources from the training data, if none found return null
row = convert_url_to_source(url)
source_encoded = row['source_encoded'].values[0]
source_bias_score = row['source_bias_score'].values[0]

In [ ]:
# Create a Pandas DataFrame from the text, avg_sentence_bias_score, source_encoded, and source_bias_score
test_pd_df = pd.DataFrame({
    'content': [text],
    'avg_sentence_bias_score': [avg_sentence_bias_score],
    'source_encoded': [source_encoded],
    'source_bias_score': [source_bias_score],
    'bias': [-1]
    })
test_pd_df

,content,avg_sentence_bias_score,source_encoded,source_bias_score,bias
0,President Donald Trump told Cabinet members la...,0.166667,29.0,1.0,-1


In [ ]:
# Pass the DataFrame and generate Sparse Matrix from the columns
X_test, y_test = preprocess_test_columns(test_pd_df, addition_features=True)
X_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 165 stored elements and shape (1, 50003)>

In [ ]:
# Predict
y_pred = int(predict(X_test, model_path)[0])

# Map the int back to leaning
if y_pred == 0:
  leaning = 'left'
elif y_pred == 1:
  leaning = 'center'
else:
  leaning = 'right'
{ 'bias': y_pred, 'leaning': leaning }

{'bias': 1, 'leaning': 'center'}